In [37]:
#Required libraries

import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import skimage
import time
import os
from skimage import data
from skimage import filters
from skimage.transform import hough_ellipse
from skimage.draw import ellipse_perimeter
from skimage import io, morphology
from skimage.morphology import disk
from skimage.filters.rank import entropy
from sklearn.metrics import confusion_matrix

In [38]:
#Template Creation for the optical disk localization

#Replace template_path with necessary filepath
template_path = r"/content/Image Segmentation/template"


template = np.ones((95,95), dtype="uint8") * 0
template = cv.circle(template, (47,47), 46, 255, -1)
cv.imwrite(os.path.join(template_path , 'temp.png'), template)

template_path = os.path.join(template_path, 'temp.png')

In [39]:
#Resizing function, extract_hardexudates() will revert back to original size
def imgResize(img):
  h = img.shape[0]
  w = img.shape[1]
  perc = 500/w
  w1 = 500
  h1 = int(h*perc)
  img_rs = cv.resize(img,(w1,h1))
  return img_rs, h, w

#Canny edge detection
def cannyEdges(img, th1, th2):
  edges = cv.Canny(img, th1, th2)
  return edges


def kmeansclust(img, k, attempts, max_iter, acc, use = 'OD'):

  if use == 'OD':
    img_rsp = img.reshape((-1,1))
  else :
    img_rsp = img.reshape((-1,3))

  img_rsp = img_rsp.astype('float32')


  criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, max_iter, acc)
  _, labels, (centers) = cv.kmeans(img_rsp, k, None, criteria, attempts, cv.KMEANS_RANDOM_CENTERS)
  centers = centers.astype('uint8')

  labels = labels.flatten()
  seg_img = centers[labels.flatten()]
  seg_img = seg_img.reshape(img.shape)
  return seg_img


#Get contours
def getContours1(img, img_main, cnt_area):
  mask1 = np.ones(img.shape, dtype="uint8") * 0
  cnts0, hier0 = cv.findContours(img.copy(),cv.RETR_LIST,cv.CHAIN_APPROX_NONE)
  for i in cnts0:
    if cv.contourArea(i) <= cnt_area:
      cv.drawContours(mask1, [i], -1, 255, -1)
  mask1 = cv.bitwise_and(img_main, img_main, mask = mask1)
  return mask1



In [40]:
#Masking optical disk but first have to retrieve its location
def extract_opticdisk(image):
  img_rs, h, w = imgResize(image)

  img_grey = cv.cvtColor(img_rs, cv.COLOR_BGR2GRAY)


  img_k = kmeansclust(img_grey, 7, 10, 400, 0.99)


  temp = cv.imread(template_path, -1)


  #TEMPLATE MATCHING
  metd = cv.TM_CCOEFF_NORMED
  temp_mat = cv.matchTemplate(img_k, temp, metd)

  min_val, max_val, min_loc, max_loc = cv.minMaxLoc(temp_mat)
  x = max_loc[0]+45
  y = max_loc[1]+45



  temp_mat = img_grey.copy()
  img_mark = cv.circle(temp_mat, (x, y) ,40, 0, -1)

  return img_mark

In [41]:

#Extraction, returns segmented BW image of the hard exudates
def extract_hardexudates(image, img_mark):

  img, h, w = imgResize(image)

  _, img_gc, _ = cv.split(img)
  clus_seg = kmeansclust(img, 8, 5, 20, 0.69, use = 'EX')
  clus_seg = cv.cvtColor(clus_seg, cv.COLOR_BGR2GRAY)
  unique, counts = np.unique(clus_seg, return_counts=True)
  _,kthm = cv.threshold(clus_seg, np.max(unique)-1, 255, cv.THRESH_BINARY)

  edges = cannyEdges(img_gc, 70, 120)
  img_cnt = cv.dilate(edges, cv.getStructuringElement(cv.MORPH_ELLIPSE,(2,2)))
  img_clean = getContours1(img_cnt, img_gc, 15)
  img_clean = cv.erode(img_clean, cv.getStructuringElement(cv.MORPH_ELLIPSE,(2,2)), iterations=1)
  img_clean = cv.dilate(img_clean, cv.getStructuringElement(cv.MORPH_ELLIPSE,(2,2)), iterations = 2)
  max_intsy = np.max(img_clean.flatten())
  img_clean[img_clean>=max_intsy] = 255
  img_clean[img_clean<max_intsy] = 0

  _, img_clean = cv.threshold(img_clean, 173, 255, cv.THRESH_BINARY)
  img_final = cv.bitwise_or(kthm, img_clean)


  img_final[img_mark==0] = 0



  img_final = cv.resize(img_final, (w,h))
  return img_final




if __name__ == "__main__":

    #Replace input and output folder with necessary filepaths
    input_folder = r"/content/Image Segmentation/input_folder"
    output_folder = r"/content/Image Segmentation/output_folder"


    if not os.path.exists(output_folder):
        os.makedirs(output_folder)


    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):

            image_path = os.path.join(input_folder, filename)
            image = cv.imread(image_path)
            img_mark = extract_opticdisk(image)

            hardexudates = extract_hardexudates(image, img_mark)


            output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_hardexudates.png")
            cv.imwrite(output_path, hardexudates)